<center><h1 style="color:maroon">Dealing with Geospatial Data and Time Series</h1>
    <img src="figures/08-geospatial-data.jpeg" style="width:1300px">
    <h3><span style="color: #045F5F">Data Science & Machine Learning for Planet Earth Lecture Series</span></h3><h6><i> by Cédric M. John <span style="size:6pts">(2022)</span></i></h6></center>

## Plan for today's Lecture 🗓 

* Spatial and temporal covariance
* Using Machine Learning to predict spatial and temporal data
* Semivariograms and geostatistics
* Geospacial interpolation with Kriging

## Intended learning outcomes 👩‍🎓

* Be able to choose the correct train-test split method
* Handling non-stationarity for machine learning modelling
* Be able to extrapolate spatial data with Kriging

# Geospatial Data
<br>

<center><img src="figures/DALLE_geospatial.png" style="width:900px;">
 © Cédric John, 2022; Image generated with <a href="https://openai.com/blog/dall-e/">DALL-E</a>
<br>Prompt: <i>A cute furry squirel with soft brown eyes uses a ruler to measure the size of the African continent, digital art</i>.</center>

### Dataset

<img src="figures/poro.png" style="padding:10px;width:500px;" align="left"/>

<span style="color:teal">**Today's dataset:** </span> We will be playing with a geographic distribution of porosity. I borrowed this dataset from my colleague <a href="https://github.com/GeostatsGuy/PythonNumericalDemos"> Prof. Michael Pyrcz on GitHub.</a> This is an ideal dataset for today as it contains data spread geographically.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("data/poro-perm.csv")  
data

#### Limiting ourselves to sand data

To simplify this example, we will only look at the sandy lithologies:

In [ ]:
sand = data[data.Facies==1]
shale = data[data.Facies==0]

In [ ]:
x = sand.X.values
y = sand.Y.values
phi = sand.Porosity.values

# Geospatial Colinearity

#### Let's plot the spatial variability of porosity

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,12))
sc = ax.scatter(x, y, c=phi,s=20);ax.set_title('Measured Porosity', size=16)
ax.set_xlabel('X Coordinates', size=14);ax.set_ylabel('Y Coordinates', size=14)
plt.colorbar(sc);ax.set_xlim(0,1000);

## Exploring relationship to distance

Let's extract all datapoints from the previous graph that are between 400 and 600 units on the <code>Y</code> axis, and average their values by their position on <code>X</code>. We will also normalize <code>X</code> (distance) and <code>Porosity</code> for easier plotting on the next slide.

In [ ]:
import matplotlib.patches as patches
fig, ax = plt.subplots(1,1, figsize=(15,12))
sc = ax.scatter(x, y, c=phi,s=20);ax.set_title('Measured Porosity', size=16)
ax.set_xlabel('X Coordinates', size=14);ax.set_ylabel('Y Coordinates', size=14)
rect = patches.Rectangle((0, 400), 1000, 200, linewidth=0, edgecolor='r', facecolor='blue', alpha=.2)
ax.plot([0,1000],[500,500],c='blue', linewidth=4, alpha=.5)
ax.add_patch(rect); plt.colorbar(sc);ax.set_xlim(0,1000);

In [ ]:
from sklearn.preprocessing import MinMaxScaler

corr_df = sand[(sand.Y>400) & (sand.Y<600)][['X','Porosity', 'Perm']]
corr_df = corr_df.groupby('X').mean().reset_index()

corr_df = pd.DataFrame(MinMaxScaler().fit_transform(corr_df).T, corr_df.columns).T
corr_df

### Let's plot the relationship between <code>Porosity</code> and <code>distance</code> on the X axis:

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ool = np.linspace(0,1,10)
ax.scatter(corr_df.X, corr_df.Porosity);ax.plot(ool,ool,c='red', linewidth=6.0, alpha=.3);
ax.set_xlabel("Normalized Distance", size=16);ax.set_ylabel("Normalized Porosity", size=16);
ax.set_xlim(0,1); ax.set_ylim(0,1);

❗ There is clearly a correlation between geographic position and values of porosity! This means that we have **spatial colinearity** (porosity is more closely related to points that are nearer to each other than far away).

## 💡 Reminder: basic assumption for train-test split

* ✂️ We split the dataset to insure our trained machine learning model **extrapolates** well to unknown samples

* 🎲 For data with spatial or temporal co-linearity, if we do a **random split** we will end up with data in our test set that is very similar (has strong colinearity) to data in our train set

* 👎 This breaks our assumption of independance of training and testing set: our assessment of the performance of our machine learning algorithm will be unrealistically high.

# Modeling Spatial Data and Time Series with Machine Learning
<br>

<center><img src="figures/DALLE_watch.png" style="width:900px;">
 © Cédric John, 2022; Image generated with <a href="https://openai.com/blog/dall-e/">DALL-E</a>
<br>Prompt: <i>An oil painting of a pocket watch in dramatic green lighting, digital art</i>.</center>

## Geographic train-test split

Often, for spatial data, the solution is to split the training and testing set based on location. This is also applicable to 1D (e.g. cores, logs) and 3D (e.g. seismic, atmospheric volume, oceans, ...) datasets. Hence, rather than take the approach shown on the left below we take the approach taken on the right:

<img src="figures/train_test_split.png" style="width:1500px">
<a href="https://en.wikipedia.org/wiki/Machine_learning_in_earth_sciences">Wikipedia</a>

👆 This only works if the data distribution in the training set is similar to the data distribution in the test set. In other words, if the data is stationary (see below).

## Train-test split for Time Series Forecasting

For time series, we simply split along the time axis: we use the older part of the time series as the training set, and the newer part as the test set for our trained algorithm:

<img src="figures/time_series_split.png" style="width:1500">
<a href="https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/">Brownlee, 2016</a>

👆 Again, this assumes that the time series has no major trends. Let's talk about **stationarity**.

## Stationarity

Stationarity is a key concept to model spatial data and to forecast time series:

<img src="figures/stationarity.png">
<a href="https://towardsdatascience.com/stationarity-in-time-series-analysis-90c94f27322">Palachy, 2019</a>

👆 So, is out data **stationary**?

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,6));ax.plot(corr_df.X, corr_df.Porosity);ax.set_xlim(0,1); ax.set_ylim(0,1);
ax.set_xlabel("Normalized Distance", size=16);ax.set_ylabel("Normalized Porosity", size=16);

### Let's try to predict permeability from porosity

And let's ignore stationarity for now (at our peril!). We will do a geographic train-test-split, keeping <span style="color:blue">**30% of the data as a test set**</span>, and <span style="color:red">**70% as our train set**</span>:

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,6));ax.plot(corr_df.X, corr_df.Porosity);ax.set_xlim(0,1); ax.set_ylim(0,1);
ax.set_xlabel("Normalized Distance", size=16);ax.set_ylabel("Normalized Porosity", size=16);
rect = patches.Rectangle((0, 0), .3, 1, linewidth=0, edgecolor='r', facecolor='blue', alpha=.2);ax.add_patch(rect);
rect = patches.Rectangle((.3, 0), 1, 1, linewidth=0, edgecolor='r', facecolor='red', alpha=.2);ax.add_patch(rect);

In [ ]:
X_test = corr_df.loc[:4,'Porosity']; y_test = corr_df.loc[:4,'Perm'];
X_train = corr_df.loc[5:,'Porosity']; y_train = corr_df.loc[5:,'Perm'];
X_test


### Let's use a <code>RandomForestRegressor</code> for our prediction:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

reg = RandomForestRegressor().fit(X_train.values.reshape(-1,1), y_train)

#### Now let's estimate the error on the test set:

In [ ]:
y_pred = reg.predict(X_test.values.reshape(-1,1))
print(f'Relative error is {mean_squared_error(y_test,y_pred)*100:.0f}% !!!!')


### Let's plot our results to explore where the errors are

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,6));ax.plot(corr_df.X, corr_df.Porosity);ax.set_xlim(0,1); ax.set_ylim(-0.2,1);
ax.scatter(corr_df.loc[:4,'X'], y_pred, c='r', marker='s', s=120);
ax.set_xlabel("Normalized Distance", size=16);ax.set_ylabel("Normalized Porosity", size=16);

### Detrending data (introducing stationarity)

We can first remove trends in the data, for instance, using a simple linear regression with distance for our variables:

In [ ]:
from sklearn.linear_model import LinearRegression

trend_poro = LinearRegression().fit(corr_df[['X']],corr_df.Porosity)
trend_perm = LinearRegression().fit(corr_df[['X']],corr_df.Perm)

In [ ]:
# Detrend:
corr_df['Poro_det'] = corr_df.Porosity - trend_poro.predict(corr_df[['X']])
corr_df['Perm_det'] = corr_df.Perm - trend_perm.predict(corr_df[['X']])

#### Let's inspect the detrended porosity:

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,6))
ax.plot(corr_df.X, corr_df.Poro_det);ax.set_xlim(0,1); ax.set_ylim(-.5,.5);
ax.set_xlabel("Normalized Distance", size=16);ax.set_ylabel("Porosity Residuals", size=16);

👉 We can now model residuals without worrying about non-stationarity of the data

### Modeling detrended data (residuals):

In [ ]:
# Train-test split:
X_test = corr_df.loc[:4,'Poro_det']; y_test = corr_df.loc[:4,'Perm_det'];
X_train = corr_df.loc[4:,'Poro_det']; y_train = corr_df.loc[4:,'Perm_det'];

# Model the residuals:
reg = RandomForestRegressor().fit(X_train.values.reshape(-1,1), y_train)

# Print prediction error:
y_pred_det = reg.predict(X_test.values.reshape(-1,1))
print(f'Relative error is {mean_squared_error(y_test,y_pred_det)*100:.0f}% !!!!')

##### We can add the trend to the residuals to obtain the true value:


In [ ]:
trend_perm.coef_

In [ ]:
y_pred_best = y_pred_det + trend_perm.coef_[0] * corr_df.loc[:4,'X']  

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,6));ax.plot(corr_df.X, corr_df.Porosity);ax.set_xlim(0,1); ax.set_ylim(-.2,1);
ax.scatter(corr_df.loc[:4,'X'], y_pred, c='r', marker='s', s=120);
ax.scatter(corr_df.loc[:4,'X'], y_pred_best, c='purple', marker='o', s=160);
ax.set_xlabel("Normalized Distance", size=16);ax.set_ylabel("Normalized Porosity", size=16);

## Modelling Time Series in Machine Learning

<img src="figures/time_series_stationarity.png" style="width:1500px">
<a href="https://towardsdatascience.com/achieving-stationarity-with-time-series-data-abd59fd8d5a0">Mitrani, 2020</a>

👉 **Same principle** as for spatial data: stationarity is a prerequisit

👉 Time Series need to be **detrended** (usually using a moving average)

👉 **Seasonality** (Short-term trends) also need to be removed to **model residuals**! This can be done with Python packages such as <code>statsmodel</a>

# Geostatistics
<br>

<center><img src="figures/DALLE_shrew.png" style="width:900px;">
 © Cédric John, 2022; Image generated with <a href="https://openai.com/blog/dall-e/">DALL-E</a>
<br>Prompt: <i>a 4k photo of a blue-gray shrew coming out of a ground hole holding a golden nugget in her hand and wearing a miners hat, digital art</i>.</center>

## Principle

In the previous section we modelled one spatial variable (<code>permeability</code>) by using another one (<code>porosity</code>). With <code>geostatistics</code> we will see that we can infer the distribution of a variable through space **using a few key assumptions**.

👆 First, let's review some key concepts

### Measuring distance

We can imagine the theoretical distribution of a continuous variable as an unknown function $Z(s)$, where $s$ is the location of the point. Any given point can be separated by a distance $h$
<img src="figures/grid_1.png" style="width:900px">
<a href="https://aegis4048.github.io/spatial-simulation-1-basics-of-variograms">Kim, 2019</a>

### Measuring distance

This distance can be measured at any span (lag) and in any direction, not just diagonally.
<img src="figures/grid_2.png" style="width:900px">
<a href="https://aegis4048.github.io/spatial-simulation-1-basics-of-variograms">Kim, 2019</a>

### Semivariogram <span style="color:teal">$\gamma(h)$</span>  (and variogram <span style="color:teal">$2\gamma$</span> )

The semivariogram (<span style="color:teal">$\gamma(h)$</span>) is half the spatial variance between two data points separated by the lag distance h:

$$\gamma(h)=\frac{1}{2n(h)}\sum_{\alpha=1}^{n(h)}(z(u\alpha)−z(u\alpha+h))^2$$

<span style="color:red">$n(h)$</span> = number of all data point pairs separated by the distance, h.

<span style="color:blue">$h$</span> = **lag distance**. Separation between two data points.

<span style="color:green">$u\alpha$</span> = data point on 2D or 3D space at the location, $\alpha$.

<span style="color:green">$u\alpha+$</span><span style="color:blue">$h$</span> = data point separated from <span style="color:green">$u\alpha$</span> by the distance, <span style="color:blue">$h$</span>.

$z($<span style="color:green">$u\alpha$</span>) = numerical value of data point

$z($<span style="color:green">$u\alpha+$</span><span style="color:blue">$h$</span>) = numerical value of data point

<img src="figures/basic_variogram.png" style="width:1300px">
<a href="https://aegis4048.github.io/spatial-simulation-1-basics-of-variograms">Kim, 2019</a>

<span style="color:purple">nugget</span>:  Variance at <span style="color:blue">$h=0$</span>

<span style="color:green">range (a)</span>: lag distance $a$ at which spatial data pairs lose correlation (i.e. reaches a the **sill**).

<span style="color:teal">Sill ($\sigma^2$)</span>: Value for the variance at the **range** distance

<span style="color:red">Variogram Model</span>: The blue datapoints are fitted with a model (the red line) which is limited to a few defined functions.

<img src="figures/basic_variogram.png" style="width:1300px">
<a href="https://aegis4048.github.io/spatial-simulation-1-basics-of-variograms">Kim, 2019</a>

#### Nugget Effect (c0)

Refers to the nonzero intercept of the variogram and is an overall estimate of error caused by measurement inaccuracy and environmental variability occurring at fine enough scales to be unresolved by the sampling interval.

### Building the semivariogram
<img src="figures/grid_3.png" style="width:1500">
<a href="https://aegis4048.github.io/spatial-simulation-1-basics-of-variograms">Kim, 2019</a>

### Building the semivariogram
<img src="figures/grid_4.png" style="width:1500">
<a href="https://aegis4048.github.io/spatial-simulation-1-basics-of-variograms">Kim, 2019</a>

### Building the semivariogram
<img src="figures/grid_5.png" style="width:1500">
<a href="https://aegis4048.github.io/spatial-simulation-1-basics-of-variograms">Kim, 2019</a>

### The variogram model

The choice of an appropriate <span style="color:red">variogram model</span> is a crucial one. The most common functions are:

#### Gaussian function
$\gamma(h)=(s-n)\left(1-\exp\left(-\frac{h^2}{r^2a}\right)\right) + n1_{(0,\infty)}(h)$

#### Spherical function
$\gamma(h)=(s-n)((\frac{3h}{2r}-\frac{h^3}{2r^3})l_{[0,r)}(h)+l_{(0,\inf)}(h))+nl_{(0,\inf)}(h)$

#### Exponential function
$\gamma(h)=(s-n)(1-e^{(-h/(ra))})+nl_{(0,\inf)}(h)$

**Other functions include** linear, power, and hole-effect (but are less commonly applied)


## Interpolating Spatial Data (Kriging)

The idea is to be able to model (interpolate) spatial data with few observations (left below) to a continuous field (right below):
<img src="figures/gold_transform.png" style="width:1500">

### The <span style="color:teal">Kriging Method</span> and the birth of <span style="color:blue">Geostatistics</span>

<img src="figures/DANIEG_KRIGE.jpg" style="height:310px;padding:3px" align="left">
<img src="figures/Georges_Matheron.jpg" style="height:330px;padding:5px" align="left">


This problem was solved first in 1951 by a South African mining Engineer (<span style="color:teal">***Danige Kriege, left***</span>) working on gold mines. It was later perfect by a French statistician (<span style="color:blue">***Georges Matheron, right***</span>) who gave birth to the field of <span style="color:blue">**Geostatistics** </span>(this term refers specifically to this technique, and not to any statistics applied in geosciences). The <span style="color:teal">**Kriging**</span> method proved so useful that  fields other than geosciences (e.g. ecology, meteorology, ...) adopted it over time.

## Simple Kriging
This is the original method invited by Krige. It relies on the assumption of stationarity for the dataset.

$$E[Z(s)]=\sum_{i=1}^n(s_i*p_i)=\mu$$

☝🏾 Assumes **stationarity** of $Z$

$$Var[Z(s)-Z(s+h)] = 2\gamma(|h|)$$

Goal of Kriging is to predict the random field $Z(s_{0})$ at location $s_{0}$ from the observed data. To do this, Kriging consists of solving the following equation:

$$\hat{Z}(s_{0})=\sum_{i=1}^{n}\beta_{i}Z(s_{i})$$

☝🏾 where the weights $\beta_{i}$ are chosen to make the estimator unbiased and of minimal prediction error.

Unbiasedness means that:
$$E[\hat{Z}(s_{0})]=E[\sum_{i=1}^{n}\beta_{i}Z(s_{i})]$$

$$=\sum_{i=1}^{n}\beta_{i}E[Z(s_{i})]$$

$$=\sum_{i=1}^{n}\beta_{i}\mu$$

Last equality derives from the stationarity of the mean. The estimator is unbiased, i.e.:

$$E[\hat{Z}(s_{0})] = E[Z(s_{0})]$$

If the weights sum to parity:

$$\sum_{i=1}^{n}\beta_i=1$$

#### Kriging Variance ($\sigma^2_{s_0}$)

$$\sigma_{s_0}\equiv E[\hat{Z}(s_0)-Z(s_0)]^2$$

$$=E[\sum_{i=1}^n\beta_i (Z(s_i)-Z(s_0))]^2$$

Which can be expanded:
$$=E[\sum_{i=1}^n\sum_{j=1}^n\beta_i \beta_j \frac{(Z(s_i)-Z(s_j))^2}{2}-\sum_{i=1}^n \beta_i(Z(s_i)-Z(s_0))^2]$$

$$=\sum_{i=1}^n\sum_{j=1}^n\beta_i \beta_j \frac{E[Z(s_i)-Z(s_j)]^2}{2}-\sum_{i=1}^n \beta_i E[Z(s_i)-Z(s_0)]^2$$

$$=\sum_{i=1}^n\sum_{j=1}^n\beta_i \beta_j \gamma(|h_{ij}|)-\sum_{i=1}^n \beta_i \gamma(|h_{i0}|)2$$

### Solving the Kriging Equation

Solving the Kriging equation means finding the best weights $\beta$ to minimize the Kriging Variance. Exactly ***how*** this is done is beyond the scope of this module (but those of you taking the ***optimization*** module might learn more about it).

Just for completeness sake, know that the method used is the **Lagrange Multiplier Method** (a constrained optimization method).

With that method, the optimal weights are found by setting to zero, the partial derivative, with respect to each $\beta_i$ of the objective function defined as:

$$f(\beta_1,...,\beta_n,\lambda)={\sigma_{s_0}}^2+2\lambda(\sum_{i=1}^n \beta_i-1))$$

Where $\lambda$ is known as the ***Lagrange Multiplier***

The partial derivative $\frac{\partial f}{\partial \lambda}$ set to zero yields the unbiasedness condition, and a system of n+1 linear equations to be solved for the n optimal weights $\beta_1, . . . , \beta_n$. The set of these linear equations is called the **ordinary kriging system (Matheron 1971)**:

$$\Bigg\{^{\sum_{j=1}^n \beta_j\gamma(|h_{ij}|) \;\;\;\;\;\; i=1,...,n}_{\sum_{j=1}^n \beta_j=1}$$

# Simple Kriging in Practice

Let's solve our spacial problem (distribution of porosity in the subsurface) using Ordinary Kriging. For this, we will use a simple library for Kriging in Python: <code>pykrige</code> (<a href="https://geostat-framework.readthedocs.io/projects/pykrige/en/stable/">documentation</a>).

In [ ]:
from pykrige.ok import OrdinaryKriging

OK = OrdinaryKriging(
    x, 
    y, 
    phi, 
    variogram_model='gaussian',
    verbose=True,
    enable_plotting=True,
    nlags=30,
)

In [ ]:
OK.variogram_model_parameters

In [ ]:
gridx = np.arange(0, 1000, 10, dtype='float64')
gridy = np.arange(0, 1000, 10, dtype='float64')
zstar, ss = OK.execute("grid", gridx, gridy)

In [ ]:
print(zstar.shape)
print(ss.shape)

In [ ]:
fig, ax_ok = plt.subplots(1,1, figsize=(15,13))
cax = ax_ok.imshow(zstar, extent=(0, 1000, 0, 1000), origin='lower')
ax_ok.scatter(x, y, c='k', marker='.'); cbar=plt.colorbar(cax)
ax_ok.set_title('Spatial Prediction of Porosity', size=18); ax_ok.set_xlim(0,1000);

## Kriging Variance

In [ ]:
fig, ax_okv = plt.subplots(1,1, figsize=(15,13))
cax = ax_okv.imshow(ss, extent=(0, 1000, 0, 1000), origin='lower')
ax_okv.scatter(x, y, c='k', marker='.'); cbar=plt.colorbar(cax)
ax_okv.set_title('Kriging Variance (estimated uncertainty) for Porosity', size=18); ax_okv.set_xlim(0,1000);

# Universal Kriging

The problem with **Ordinary Kriging** is that it assumes stationarity. Of course, most geospatial applications are **NOT** stationary. Universal Kriging is meant to solve this problem. The main difference is that it models the predictor variable ($Z(s)$) as a linear combination of a deterministic term ( the drift "$\mu(s)$", non-stationary) and a probabilistic term ("$Y(s)$", stationary):

$$Z(s)=\mu(s)+Y(s)$$

Suppose the drift μ(s) can be represented as a linear combination of known functions $\{fl(s),\;\;\;\; l = 1, . . . , k\}$, with unknown coefficients $\{\beta l\}$,
$$\mu(s)=\sum_{l=1}^k \beta_l f_l(s)$$

This means that the mean and covariance can be expressed as:
$$E[Z(s)]=\sum_{l=1}^k \beta_l f_l(s)$$

$$E[(Z(s_1)-\mu(s_1))(Z(s_2)-\mu(s_2))]\equiv E[Y(s_1)Y(s_2)]$$
$$= Cov(s1-s2)$$

The Universal Kriging predictor has the same form as for Ordinary Kriging:
$$\hat{Z}(s_{0})=\sum_{i=1}^{n}\beta_{i}Z(s_{i})$$

But the unbiasedness condition becomes more complicated (Matheron, 1969):
$$f_l(s_0)=\sum_{i=1}^n \beta_1 f_l(s_i) \;\;\;\;\;\; l=1,...,k$$

The Universal Kriging Variance becomes:
$$=\sum_{i=1}^n\sum_{j=1}^n\beta_i \beta_j Cov(s_i-s_j)-2\sum_{i=1}^n \beta_i Cov(s_i-s_0) + var(Z(s_0))$$

The $n+k$ Universal Kriging linear equations are:
$$\Bigg\{^{\sum_{j=1}^n \beta_j Cov(s_i-S_j)+\lambda_l f_l(s_i) = Cov(s_i-s_0) \;\;\;\;\;\; i=1,...,n}_{\sum_{j=1}^n \beta_j f_l(s_j)=f_l(s_0)\;\;\;\;\;\; i=1,...,k}$$

☝🏻 I skipped several mathematical steps here. If you are interested in the details, read <a href="https://link.springer.com/book/10.1007/0-387-35429-8">Le and Zidek, 2006, Statistical Analysis of Environmental Space-Time Processes (Chapter 7)</a>

In [ ]:
from pykrige.uk import UniversalKriging

UK = UniversalKriging(
    x, 
    y, 
    phi, 
    variogram_model='gaussian',
    verbose=True,
    enable_plotting=True,
    nlags=30,
)

In [ ]:
gridx = np.arange(0, 1000, 10, dtype='float64')
gridy = np.arange(0, 1000, 10, dtype='float64')
zstar_uk, ss_uk = UK.execute("grid", gridx, gridy)

In [ ]:
fig, ax_uk = plt.subplots(1,1, figsize=(15,13))
cax = ax_uk.imshow(zstar_uk, extent=(0, 1000, 0, 1000), origin='lower')
ax_uk.scatter(x, y, c='k', marker='.'); cbar=plt.colorbar(cax)
ax_uk.set_title('Universal Kriging Porosity estimate', size=18); ax_uk.set_xlim(0,1000);

In [ ]:
fig, ax_ukv = plt.subplots(1,1, figsize=(15,13))
cax = ax_ukv.imshow(ss_uk, extent=(0, 1000, 0, 1000), origin='lower')
ax_ukv.scatter(x, y, c='k', marker='.'); cbar=plt.colorbar(cax)
ax_ukv.set_title('Universal Kriging Variance for Porosity', size=18); ax_ukv.set_xlim(0,1000);

## Limitations of Kriging

🔺 **Weights of the kriging interpolator depend on the modeled variogram** <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;👉  kriging  quite sensitive to misspecification of the variogram model.

🔺 **Assumptions of the kriging model** (e.g. that of second-order stationarity) may be difficult to meet in the context of many environmental exposures<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;👉  some newer methods (e.g. Bayesian approaches) have  been developed to try and surmount these obstacles

🔺 **Accuracy of interpolation by kriging** will be limited if the number of sampled observations is small, the data is limited in spatial scope, or if the data are not sufficiently spatially correlated. 

# Suggested Resources

## 📺 Videos 
* 📼 <a href="https://youtu.be/J-IB4_QL7Oc">The Kriging Model : Data Science Concepts</a> by Ritvik Math, 2020
* 📼 <a href="https://youtu.be/QGjdS1igq78">Machine-learning based modelling of spatial and spatio-temporal data (introduction)</a>, Lecture by Hanna Meyer, Prague Summer School, 2018

## 📚 Further Reading 
* 📖 <a href="https://gisgeography.com/kriging-interpolation-prediction/">Kriging Interpolation – The Prediction Is Strong in this One</a> by GISGeography, 2022
* 📖 <a href="https://kevinkotze.github.io/ts-6-unit-roots/">Non-stationarity</a> by Kevin Kotzé (a bit in depth but a lot of good references
* 📖 <a href="https://www.analyticsvidhya.com/blog/2021/03/introducing-machine-learning-for-spatial-data-analysis/#:~:text=Machine%20Learning%20for%20spatial%20data%20analysis%20builds%20a%20model%20to,the%20spatial%20attribute%20into%20account.">Introducing Machine Learning for Spatial Data Analysis</a> by Rendik, 2021
* 📖 <a href="https://link.springer.com/book/10.1007/0-387-35429-8">Statistical Analysis of Environmental Space-Time Processes (Book)</a> by Le and Zidek, 2006
